In [1]:
gestures = ["open_palm",
             "open_dorsal",
             "fist_palm",
             "fist_dorsal",
             "three_fingers_palm",
             "three_fingers_dorsal"]

In [2]:
%matplotlib widget

In [3]:
import ipywidgets as widgets
import cv2
import numpy as np
from ipywebrtc import CameraStream, ImageRecorder, VideoRecorder
#from src.View import JupyterGUIFactory
#from src.Controller import StateManager
#from src.landmark_names import landmark_names
from IPython.display import clear_output
import tensorflow as tf
import pandas as pd
from numpy import genfromtxt
import os
import matplotlib.pyplot as plt
import cv2
import time
import base_version as base
import modified_version as mod


In [4]:
from pandas import read_excel
# find your sheet name at the bottom left of your excel file and assign 
# it to my_sheet 
my_sheet = 'Tabelle1' # change it to your sheet name
file_name = '../data/Video Data.xlsx' # change it to the name of your excel file
df = read_excel(file_name, sheet_name = my_sheet)
df.head() # shows headers with top 5 rows

,ID,source,frame,camera_facing_side,gesture,palm_root_x,palm_root_y,palm_thumb_1_x,palm_thumb_1_y,palm_thumb_2_x,...,dorsal_ring_4_x,dorsal_ring_4_y,dorsal_pinky_1_x,dorsal_pinky_1_y,dorsal_pinky_2_x,dorsal_pinky_2_y,dorsal_pinky_3_x,dorsal_pinky_3_y,dorsal_pinky_4_x,dorsal_pinky_4_y
0,102,102/open_palm.webm,0,open,palm,279,369,189,332,137,...,0,0,0,0,0,0,0,0,0,0
1,102,102/open_palm.webm,1,open,palm,279,370,188,331,137,...,0,0,0,0,0,0,0,0,0,0
2,102,102/open_palm.webm,2,open,palm,279,370,187,331,137,...,0,0,0,0,0,0,0,0,0,0
3,102,102/open_palm.webm,3,open,palm,278,370,186,330,136,...,0,0,0,0,0,0,0,0,0,0
4,102,102/open_palm.webm,4,open,palm,278,371,185,329,136,...,0,0,0,0,0,0,0,0,0,0


In [63]:
def split_into_images(df, each_id, video_name ,row):
    path="../videos/"+str(each_id) + "/" + video_name+ ".webm"
    vid=str(each_id) + "/" + video_name+ ".webm"
    foo = cv2.VideoCapture(path)
    ret, frame = foo.read()
    
    counter = 0
    while ret:
        ret, frame = foo.read()
        if not ret:
            break
        pts=base.hand_extraction(frame, 22)
        print(counter)
        cols=0
        for i in range(len(pts)):
           
            if "palm" in video_name:
                cols= df.columns[5:45]
              
            elif "dorsal" in video_name:
                cols= df.columns[45:]
            
            if pts[i][0]==0 or pts[i][0]==2:

                #df[(df['source']==vid) & (df['frame'] == counter)][cols[pts[i][0]]]= pts[i][1]
                #df[(df['source']==vid) & (df['frame'] == counter)][cols[pts[i][0]]+1]= pts[i][2]
                
                df.loc[row,cols[pts[i][0]]]= pts[i][1]
                df.loc[row,cols[pts[i][0]+1]]= pts[i][2]
              
          
            elif pts[i][0]!=1 and pts[i][0]<21:
                   
                #df[(df['source']==vid) & (df['frame'] == counter)][2*cols[pts[i][0]]-1]= pts[i][1]
                #df[(df['source']==vid) & (df['frame'] == counter)][2*cols[pts[i][0]]]= pts[i][2]
                df.loc[row,cols[2*pts[i][0]-2]]= pts[i][1]
                df.loc[row, cols[2*pts[i][0]-1]]= pts[i][2]
               
                
        #print(row)
        row+=1
        #if row==1709:
        #    break
        counter += 1
    foo.release()
    return df, row

In [64]:
df2=df.copy()
df2,row=split_into_images(df2, 102, "open_palm",0 )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [65]:
df[(df['source']=="102/open_palm.webm") ]

,ID,source,frame,camera_facing_side,gesture,palm_root_x,palm_root_y,palm_thumb_1_x,palm_thumb_1_y,palm_thumb_2_x,...,dorsal_ring_4_x,dorsal_ring_4_y,dorsal_pinky_1_x,dorsal_pinky_1_y,dorsal_pinky_2_x,dorsal_pinky_2_y,dorsal_pinky_3_x,dorsal_pinky_3_y,dorsal_pinky_4_x,dorsal_pinky_4_y
0,102,102/open_palm.webm,0,open,palm,273,390,180,318,129,...,0,0,0,0,0,0,0,0,0,0
1,102,102/open_palm.webm,1,open,palm,273,390,180,318,128,...,0,0,0,0,0,0,0,0,0,0
2,102,102/open_palm.webm,2,open,palm,274,391,180,318,128,...,0,0,0,0,0,0,0,0,0,0
3,102,102/open_palm.webm,3,open,palm,283,391,180,318,128,...,0,0,0,0,0,0,0,0,0,0
4,102,102/open_palm.webm,4,open,palm,273,391,170,318,128,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,102,102/open_palm.webm,341,open,palm,273,395,174,331,125,...,0,0,0,0,0,0,0,0,0,0
342,102,102/open_palm.webm,342,open,palm,273,396,174,333,125,...,0,0,0,0,0,0,0,0,0,0
343,102,102/open_palm.webm,343,open,palm,273,396,174,335,125,...,0,0,0,0,0,0,0,0,0,0
344,102,102/open_palm.webm,344,open,palm,272,396,174,337,125,...,0,0,0,0,0,0,0,0,0,0


In [66]:
df2[(df2['source']=="102/open_palm.webm") ]

,ID,source,frame,camera_facing_side,gesture,palm_root_x,palm_root_y,palm_thumb_1_x,palm_thumb_1_y,palm_thumb_2_x,...,dorsal_ring_4_x,dorsal_ring_4_y,dorsal_pinky_1_x,dorsal_pinky_1_y,dorsal_pinky_2_x,dorsal_pinky_2_y,dorsal_pinky_3_x,dorsal_pinky_3_y,dorsal_pinky_4_x,dorsal_pinky_4_y
0,102,102/open_palm.webm,0,open,palm,273,390,180,318,129,...,0,0,0,0,0,0,0,0,0,0
1,102,102/open_palm.webm,1,open,palm,273,390,180,318,128,...,0,0,0,0,0,0,0,0,0,0
2,102,102/open_palm.webm,2,open,palm,274,391,180,318,128,...,0,0,0,0,0,0,0,0,0,0
3,102,102/open_palm.webm,3,open,palm,283,391,180,318,128,...,0,0,0,0,0,0,0,0,0,0
4,102,102/open_palm.webm,4,open,palm,273,391,170,318,128,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,102,102/open_palm.webm,341,open,palm,273,401,170,328,118,...,0,0,0,0,0,0,0,0,0,0
342,102,102/open_palm.webm,342,open,palm,273,391,169,318,118,...,0,0,0,0,0,0,0,0,0,0
343,102,102/open_palm.webm,343,open,palm,273,401,170,328,118,...,0,0,0,0,0,0,0,0,0,0
344,102,102/open_palm.webm,344,open,palm,273,401,169,328,118,...,0,0,0,0,0,0,0,0,0,0


In [7]:
import cv2
import time
import base_version as base
import modified_version as mod

for i in range(59):
    path = '../img/img_{}.png'
    print(base.hand_extraction(cv2.imread(path.format(i)), 22))


[(0, 315, 318), (1, 273, 266), (2, 232, 224), (3, 221, 203), (4, 274, 141), (5, 263, 162), (6, 263, 193), (7, 263, 213), (8, 324, 141), (9, 304, 192), (10, 294, 213), (11, 293, 234), (12, 356, 161), (13, 325, 203), (14, 325, 224), (15, 314, 255), (16, 386, 192), (17, 366, 203), (18, 356, 234), (19, 356, 245)]
[(0, 314, 296), (1, 355, 255), (2, 396, 214), (3, 407, 182), (4, 377, 172), (5, 376, 161), (6, 376, 161), (7, 366, 193), (8, 356, 203), (9, 345, 151), (10, 355, 193), (11, 345, 203), (12, 335, 203), (13, 314, 161), (14, 325, 213), (15, 324, 213), (16, 305, 193), (17, 284, 172), (18, 304, 213), (19, 294, 213), (20, 284, 193)]
[(0, 335, 422), (1, 314, 359), (2, 284, 297), (3, 283, 234), (4, 263, 192), (5, 376, 245), (6, 407, 171), (7, 418, 120), (8, 438, 78), (9, 408, 276), (10, 459, 192), (11, 500, 150), (12, 531, 119), (13, 438, 307), (14, 510, 244), (15, 542, 203), (16, 573, 172), (17, 459, 339), (18, 511, 297), (19, 542, 276), (20, 573, 255)]
[(0, 283, 391), (1, 232, 339), (2, 1

KeyboardInterrupt: 

In [71]:
def get_ids(df):
    return df['ID'].unique()

In [72]:
get_ids(df)

array([102, 159, 294, 441, 564, 576, 609, 666, 711, 723], dtype=int64)

In [73]:
def landmark_prediction(df):
    df_predicted=df.copy()
    gestures = ["open_palm",
             "open_dorsal",
             "fist_palm",
             "fist_dorsal",
             "three_fingers_palm",
             "three_fingers_dorsal"]
    row=0
    for each_id in get_ids(df):
        for video_name in gestures:
             df_predicted, row =split_into_images(df_predicted,each_id, video_name, row )
                
    print(row)
    return df_predicted

In [ ]:
df_predicted=landmark_prediction(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110


In [67]:
def evaluate(df, df_predicted, threshold):
    correct=0
    wrong=0
    for row in range(df.shape[0]):
        for landmark in df.columns[5:]:
            if  df.loc[row, landmark]== 0:
                dummy=1
            elif abs(df_predicted.loc[row, landmark] - df.loc[row, landmark] ) < threshold:
                correct+=1
            else:
                wrong+=1
    print ("The model of landmark extraction accuracy within acceptable margin of " +str (threshold ) + " is %"+ str(correct*100/(correct+wrong)))
    return correct, wrong
                
       
   

In [68]:
correct, wrong= evaluate(df, df2, 50)

The model of landmark extraction accuracy within acceptable margin of 50 is %99.24576802342033


In [ ]:
correct, wrong=evaluate(df, df_predicted, 100)

In [ ]:
correct, wrong= evaluate(df, df_predicted, 50)